In [ ]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture
from tqdm import tqdm
import math
import cmath
from scipy.integrate import simpson, romb
from scipy.stats import iqr, norm, gaussian_kde
from scipy.special import erfc, wofz
import pickle
import os

In [ ]:
def P_x_theta(x, theta, a, r2, p = 0.6810463):

    tau = np.sqrt(1-r2)
    beta = np.abs(r2)
    W0 = 1 / np.sqrt(np.pi) / beta * np.exp(- x**2 / beta**2)

    c0 = tau / (np.sqrt(tau**2 + a**2))
    c1 = a / (np.sqrt(tau**2 + a**2))
    mu = r2 * np.cos(theta)
    correct_term = (1 - p * c1**2 + 2 * p * c1**2 * x**2 / beta**2 + 2 * np.sqrt(2) * p * c0 * c1 * mu * x / beta)

    # correct_term = 1
    Px = W0 * correct_term 

    return Px

def Laguerre_poly(n, X):
    summ = 0
    for k in range(n+1):
        summ += ((-1)**k * math.comb(n, k) * X**k) / math.factorial(k)
    return summ

def transition_kernel(X, mu, nu, n, alpha):
    term1 = 1 / 2 / np.pi * np.exp(- (nu**2 + mu**2) / 4)
    term2_power = 1j * X + (nu - 1j * mu) / np.sqrt(2) * np.conjugate(alpha) - (nu + 1j * mu) / np.sqrt(2) * alpha
    term2 = np.exp(term2_power)
    term3 = Laguerre_poly(n, (nu**2 + mu**2) / 2)
    return term1 * term2 * term3

def integrand(X, MU, NU, n, a, alpha, r2, p):
    TH = np.arctan2(NU, MU)
    return P_x_theta(X, TH, a, r2, p) * transition_kernel(X, MU, NU, n, alpha)

def cube_integral(n, a, alpha, r2, Lx, Lmu, Lnu, Nx, Nmu, Nnu, p=0.6810463):
    X = np.linspace(-Lx,  Lx,  Nx)
    MU = np.linspace(-Lmu, Lmu, Nmu)
    NU = np.linspace(-Lnu, Lnu, Nnu)

    # broadcast to 3D grid
    X3  = X[:, None, None]
    MU3 = MU[None, :, None]
    NU3 = NU[None, None, :]

    F = integrand(X3, MU3, NU3, n, a, alpha, r2, p)

    # integrate real and imaginary parts separately
    Ireal = np.trapz(np.trapz(np.trapz(F.real, NU, axis=2), MU, axis=1), X, axis=0)
    Iimag = np.trapz(np.trapz(np.trapz(F.imag, NU, axis=2), MU, axis=1), X, axis=0)
    return Ireal + 1j*Iimag




In [ ]:
a = 0.5
alpha = 0.3
r2 = 0.925
Lx = 4
Lmu = 10
Lnu = 10
Nx = 100
Nmu = 50
Nnu = 50


n = 3

alpha_values = np.linspace(-4, 4, 100)
wn_alpha_list = []
for alpha in tqdm(alpha_values):
    val = cube_integral(n, a, alpha, r2, Lx, Lmu, Lnu, Nx, Nmu, Nnu)
    wn_alpha_list.append(val)


In [ ]:
linewidth = 2

plt.plot(alpha_values, np.real(wn_alpha_list), linewidth = linewidth, label=r'$Re[w_n(\alpha)]$')
plt.plot(alpha_values, np.imag(wn_alpha_list), linewidth = linewidth, label=r'$Im[w_n(\alpha)]$')

plt.xlabel(r'$\alpha$')
plt.ylabel(r'$w_n(\alpha)$')
plt.legend()
plt.title('n = {}'.format(n))

In [ ]:
def data_wrapper(theta_list, x_list, N_theta):
    grouped_data_dict = {i: [] for i in range(N_theta)}

    theta_mod = np.mod(theta_list, np.pi)
    theta_even_odd = np.floor(theta_list / np.pi).astype(int) % 2
    bin_width = np.pi / N_theta
    bin_indices = np.floor(theta_mod / bin_width).astype(int)

    # bin_indices = np.clip(bin_indices, 0, N_theta - 1)

    for bi, x, eo in zip(bin_indices, x_list, theta_even_odd):
        grouped_data_dict[int(bi)].append(float(x) * ((-1) ** (eo+1)))

    grouped_theta_list = []
    for i in range(N_theta):
        grouped_theta_list.append((i + 0.5) * bin_width)

    return grouped_theta_list, grouped_data_dict


def pdf_kernel_optimal_bandwidth_histogram(X, X_samples, kernel_method, bandwidth_method):

    """
    kernel_method: 'gaussian' or 'epanechnikov'
    bandwidth_method: 'cv' for cross-validation or 'silverman' for Silverman's rule of thumb
    """

    if bandwidth_method == 'cv':
        fold = 5
        bw_precisions = 30
        bandwidths = np.logspace(-1, 1, bw_precisions)  # Testing range of bandwidths
        grid = GridSearchCV(KernelDensity(kernel=kernel_method), {'bandwidth': bandwidths}, cv=fold)  # 5-fold cross-validation
        grid.fit(X_samples[:, None])
        bd = grid.best_params_['bandwidth']
    
    elif bandwidth_method == 'silverman':
        silverman_bandwidth = 0.91 * min(np.std(X_samples, ddof = 1), iqr(X_samples) / 1.34) * len(X_samples) ** (-1 / 5)
        bd = silverman_bandwidth

    kde = KernelDensity(kernel=kernel_method, bandwidth=bd).fit(X_samples[:, None])
    w_values = np.exp(kde.score_samples(X.reshape(-1,1)))

    return w_values, bd




In [ ]:
theta_list, x_list = np.loadtxt("DataPhase28.dat", unpack=True)
T = len(theta_list)


n_precision = 100
N_theta = 20
X_precision = np.linspace(-Lx, Lx, n_precision)

grouped_theta_list, grouped_data_dict = data_wrapper(theta_list, x_list, N_theta)



In [ ]:
def cube_integral_experimental(n, a, alpha, r2, X_list, mu_list, nu_list, p=0.6810463):
    X = X_list
    MU = mu_list
    NU = nu_list

    # broadcast to 3D grid
    X3  = X[:, None, None]
    MU3 = MU[None, :, None]
    NU3 = NU[None, None, :]

    F = integrand(X3, MU3, NU3, n, a, alpha, r2, p)

    # integrate real and imaginary parts separately
    Ireal = np.trapz(np.trapz(np.trapz(F.real, NU, axis=2), MU, axis=1), X, axis=0)
    Iimag = np.trapz(np.trapz(np.trapz(F.imag, NU, axis=2), MU, axis=1), X, axis=0)
    return Ireal + 1j*Iimag

In [ ]:
trial_bin = 1

xb = np.array(grouped_data_dict[trial_bin])
W_Gaussian, bw_Gaussian = pdf_kernel_optimal_bandwidth_histogram(X_precision, xb, kernel_method = 'gaussian', bandwidth_method = 'silverman')

theta = grouped_theta_list[trial_bin]

radius = np.linspace(-8,8,200)
mu_list = radius * np.cos(theta)
nu_list = radius * np.sin(theta)

n = 3

alpha_values = np.linspace(-4, 4, 100)
wn_alpha_list = []
for alpha in tqdm(alpha_values):
    val = cube_integral_experimental(n, a, alpha, r2, X_precision, mu_list, nu_list)
    wn_alpha_list.append(val)

In [ ]:

plt.plot(alpha_values, np.real(wn_alpha_list), linewidth = linewidth, label=r'$Re[w_n(\alpha)]$')

In [ ]:
nu_list